In [10]:
import pandas as pd
import numpy as np

In [2]:
def cusum_events(df: pd.DataFrame,
                 h: float=None,
                 span: int=100,
                 devs: float=2.5) -> pd.DataFrame:
    '''    
    Compute CUSUM events for a given price series.    
    Args:        
    df (pd.DataFrame): Dataframe with price time series                           
    in a single column.        
    h (float): Arbitrary cumulative returns value limit to trigger                   
    the CUSUM filter. 
    The filter is symmetric. 
    If h is None exponentially weighted standard deviation will                   
    be used.        
    span (int): Span for exponential weighting of standard deviation.        
    devs (float): Standard deviations to compute variable                       
    trigger limits if h is not defined.
    Returns:        
    pd.DataFrame: Dataframe containing differentiated series.    
    '''    
    # Events e:
    e = pd.DataFrame(0, index=df.index,
                     columns=['CUSUM_Event'])
    s_pos = 0
    s_neg = 0
    r = df.pct_change()
    
    for idx in r.index:
        if h is None:
            h_ = r[:idx].ewm(span=span).std().values[-1][0]*devs
        else: h_ = h
        s_pos = max(0, s_pos+r.loc[idx].values)
        s_neg = min(0, s_neg+r.loc[idx].values)        
        if s_neg < -h_:
            s_neg = 0
            e.loc[idx] = -1 
        elif s_pos > h_:
            s_pos = 0
            e.loc[idx] = 1
    
    return e

In [ ]:
This function takes a price series dataframe and an arbitrary returns limit h to calculate fixed events. If no h limit is defined, span and devs will control the ewm and the deviations to generate events.


Let us get some very common close prices for our testing and checking of the function:

self = QuantBook()
spy = self.AddEquity('SPY').Symbol
bonds = self.AddEquity('TLT').Symbol
start = datetime(2010,1,1)
end = datetime(2020,1,1)
spy_close = self.History(spy, start, end, Resolution.Daily).unstack(level=0)['close']
bonds_close = self.History(bonds, start, end, Resolution.Daily).unstack(level=0)['close']

In [42]:
def getTEvents(gRaw,h):
    tEvents = []
    sPos = 0
    sNeg = 0
    diff=gRaw.diff()
    for i in diff.index[1:]:
        sPos = max(0,sPos+diff.loc[i][0])
        sNeg = min(0,sNeg+diff.loc[i][0])
        if sNeg<-h:
            sNeg=0;
            tEvents.append(i)
        elif sPos>h:
            sPos=0;
            tEvents.append(i)
    # return pd.DatetimeIndex(tEvents)
    return tEvents

In [44]:
gRaw = v 
getTEvents(gRaw,0.2)

[1, 4, 7, 9]

In [ ]:
v

In [25]:
tEvents = []
sPos = 0
sNeg = 0
diff=gRaw.diff().fillna(0)
diff

,0
0,0.000000
1,-0.436992
2,-0.195239
3,0.051552
4,0.193897
5,0.191819
6,-0.152077
7,0.203788
8,0.024416
9,-0.225164


In [30]:
diff.loc[i][0]

-0.4369916510575701

In [31]:
for i in diff.index[1:]:
    sPos = max(0,sPos+diff.loc[i][0])
    sNeg = min(0,sNeg+diff.loc[i][0])
   
    if sNeg<-h:
        sNeg=0;tEvents.append(i)
    elif sPos>h:
        sPos=0;tEvents.append(i)

NameError: name 'h' is not defined